# 📊 Data Cleaning Tutorial with Polars

## หลักสูตรการทำความสะอาดข้อมูลด้วย Polars DataFrame

ในหลักสูตรนี้เราจะเรียนรู้การทำความสะอาดข้อมูล (Data Cleaning) อย่างเป็นระบบด้วย Polars ซึ่งเป็น DataFrame library ที่เร็วและมีประสิทธิภาพสูง

### 🎯 วัตถุประสงค์การเรียนรู้
- เข้าใจหลักการ Data Cleaning ที่สำคัญ
- เรียนรู้การใช้ Polars สำหรับการจัดการข้อมูล
- สามารถแก้ไขปัญหาข้อมูลที่พบบ่อยได้
- รู้จักเทคนิคการตรวจสอบคุณภาพข้อมูล

### 📋 สารบัญ
1. **การเตรียมข้อมูล** - การโหลดและสำรวจข้อมูล
2. **การตรวจสอบข้อมูล** - การหาปัญหาและข้อมูลที่ผิดปกติ
3. **การจัดการ Missing Values** - การแก้ไขข้อมูลที่หายไป
4. **การแก้ไขข้อมูลที่ผิดปกติ** - การจัดการ Outliers และข้อมูลผิด
5. **การแปลงข้อมูล** - การเปลี่ยนประเภทข้อมูลและรูปแบบ
6. **การตรวจสอบผลลัพธ์** - การยืนยันความถูกต้องของข้อมูล


In [ ]:
# Import libraries ที่จำเป็น
import polars as pl
import numpy as np
# ไม่ต้องใช้ pandas เพราะใช้ Polars แทน
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ตั้งค่า Polars เพื่อแสดงผลได้ดีขึ้น
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(10)

print("✅ Libraries โหลดสำเร็จแล้ว!")
print(f"Polars version: {pl.__version__}")
print(f"NumPy version: {np.__version__}")


## 🏗️ Hongsa Drilling Database - ข้อมูลจริง

### ข้อมูลการเจาะสำรวจเหมืองถ่านหิน
- **ไฟล์ต้นฉบับ**: DH70.xlsx
- **Worksheet**: DAT201  
- **ประเภทข้อมูล**: Diamond Core Drilling
- **จำนวนข้อมูล**: 15,400+ แถว, 34 คอลัมน์
- **จำนวนหลุมเจาะ**: 70 หลุม

### โครงสร้างข้อมูลที่คาดหวัง:
1. **Collars** (ปากหลุมเจาะ) - มี coordinates (easting > 100000)
2. **Lithology Logs** (ชั้นหิน) - มี depth intervals (depth < 1000)  
3. **Sample Analyses** (ผลวิเคราะห์) - มีข้อมูล analysis (IM, TM, Ash, etc.)
4. **Rock Types** (ประเภทหิน) - lookup table
5. **Seam Codes** (รหัสชั้นถ่าน) - lookup table


In [ ]:
# อ่านข้อมูลจาก DH70.xlsx
import openpyxl

# อ่านไฟล์ Excel
wb = openpyxl.load_workbook('DH70.xlsx', data_only=True)
ws = wb['DAT201']

print(f"📊 ข้อมูล Excel:")
print(f"  - จำนวนแถว: {ws.max_row}")
print(f"  - จำนวนคอลัมน์: {ws.max_column}")

# อ่านข้อมูลทั้งหมด
data = []
for row_num in range(1, ws.max_row + 1):
    row_data = []
    for col_num in range(1, ws.max_column + 1):
        cell_value = ws.cell(row=row_num, column=col_num).value
        row_data.append(cell_value)
    data.append(row_data)

print(f"✅ อ่านข้อมูลสำเร็จ: {len(data)} แถว")


In [ ]:
# ทำความสะอาดข้อมูลและแปลงเป็น Polars DataFrame
headers = data[0]
print(f"📋 Headers: {len(headers)} คอลัมน์")
print(f"ตัวอย่าง headers: {headers[:10]}")

# เตรียมข้อมูลสำหรับ Polars
rows = []
for row in data[1:]:
    # ทำความสะอาดแถว
    cleaned_row = []
    for cell in row:
        if cell is None or cell == '':
            cleaned_row.append(None)
        elif cell == -1:
            cleaned_row.append(None)  # แปลง -1 เป็น NULL
        elif isinstance(cell, str) and cell.startswith('=IF('):
            cleaned_row.append(None)  # แปลงสูตร Excel เป็น NULL
        else:
            cleaned_row.append(cell)
    rows.append(cleaned_row)

# สร้าง Polars DataFrame
column_names = [f"col_{i}" for i in range(len(headers))]
df = pl.DataFrame(rows, schema=column_names)

print(f"✅ สร้าง Polars DataFrame สำเร็จ:")
print(f"  - ขนาด: {df.shape}")
print(f"  - คอลัมน์: {len(df.columns)}")


In [ ]:
# ตรวจสอบข้อมูลพื้นฐาน
print("🔍 ตรวจสอบข้อมูลพื้นฐาน:")
print("=" * 50)

# แสดง 5 แถวแรก
print("📊 5 แถวแรก:")
print(df.head())

# ตรวจสอบข้อมูลที่หายไป
print("\n❓ ข้อมูลที่หายไป:")
missing_data = df.null_count()
print(missing_data)

# ตรวจสอบข้อมูลซ้ำ
print(f"\n🔄 ข้อมูลซ้ำ: {df.duplicated().sum()} แถว")

# ตรวจสอบประเภทข้อมูล
print("\n📋 ประเภทข้อมูล:")
print(df.dtypes)


In [ ]:
# แยกข้อมูล Collars (ปากหลุมเจาะ)
# Collars มี easting > 100000 (coordinates)
collars = df.filter(
    (pl.col("col_0").is_not_null()) & 
    (pl.col("col_1") > 100000) &  # easting
    (pl.col("col_2") > 100000)    # northing
)

print(f"🏗️ Collars (ปากหลุมเจาะ): {len(collars)} หลุม")
print("ตัวอย่างข้อมูล Collars:")
print(collars.head())

# ตรวจสอบข้อมูล Collars
if len(collars) > 0:
    print(f"\n📊 สถิติ Collars:")
    print(f"  - Easting: {collars['col_1'].min():.0f} - {collars['col_1'].max():.0f}")
    print(f"  - Northing: {collars['col_2'].min():.0f} - {collars['col_2'].max():.0f}")
    print(f"  - Elevation: {collars['col_3'].min():.1f} - {collars['col_3'].max():.1f}")
    print(f"  - Total Depth: {collars['col_6'].min():.1f} - {collars['col_6'].max():.1f}")


In [ ]:
# แยกข้อมูล Sample Analyses (ผลการวิเคราะห์)
# Samples มีข้อมูล analysis columns (col_12 ถึง col_22)
samples = df.filter(
    (pl.col("col_0").is_not_null()) &
    (pl.col("col_12").is_not_null() |  # IM
     pl.col("col_13").is_not_null() |  # TM
     pl.col("col_14").is_not_null())   # Ash
)

print(f"🧪 Sample Analyses: {len(samples)} ตัวอย่าง")
print("ตัวอย่างข้อมูล Samples:")
print(samples.head())

# ตรวจสอบข้อมูล Samples
if len(samples) > 0:
    print(f"\n📊 สถิติ Samples:")
    print(f"  - IM: {samples['col_12'].min():.2f} - {samples['col_12'].max():.2f}")
    print(f"  - TM: {samples['col_13'].min():.2f} - {samples['col_13'].max():.2f}")
    print(f"  - Ash: {samples['col_14'].min():.2f} - {samples['col_14'].max():.2f}")
    print(f"  - Gross CV: {samples['col_18'].min():.0f} - {samples['col_18'].max():.0f}")


In [ ]:
# แยกข้อมูล Lithology Logs (ชั้นหิน)
# Lithology มี depth < 1000 และ col_1 < col_2
lithology = df.filter(
    (pl.col("col_0").is_not_null()) &
    (pl.col("col_1") < 1000) &      # depth จาก
    (pl.col("col_2") < 1000) &      # depth ถึง
    (pl.col("col_1") != pl.col("col_2"))  # จาก != ถึง
)

print(f"🪨 Lithology Logs: {len(lithology)} ช่วง")
print("ตัวอย่างข้อมูล Lithology:")
print(lithology.head())

# ตรวจสอบข้อมูล Lithology
if len(lithology) > 0:
    print(f"\n📊 สถิติ Lithology:")
    print(f"  - Depth From: {lithology['col_1'].min():.1f} - {lithology['col_1'].max():.1f}")
    print(f"  - Depth To: {lithology['col_2'].min():.1f} - {lithology['col_2'].max():.1f}")
    print(f"  - Thickness: {lithology['col_3'].min():.1f} - {lithology['col_3'].max():.1f}")
    
    # ตรวจสอบ Rock Types
    rock_types = lithology['col_4'].unique().drop_nulls()
    print(f"  - Rock Types: {len(rock_types)} ชนิด")
    print(f"  - ตัวอย่าง Rock Types: {rock_types.head().to_list()}")


In [ ]:
# สรุปข้อมูลทั้งหมด
print("📊 สรุปข้อมูล Hongsa Drilling Database:")
print("=" * 60)
print(f"📁 ข้อมูลต้นฉบับ: {len(df)} แถว")
print(f"🏗️ Collars (ปากหลุม): {len(collars)} หลุม")
print(f"🧪 Sample Analyses: {len(samples)} ตัวอย่าง")
print(f"🪨 Lithology Logs: {len(lithology)} ช่วง")

# ตรวจสอบความครอบคลุมของข้อมูล
total_holes = len(collars)
holes_with_lithology = len(lithology['col_0'].unique())
holes_with_samples = len(samples['col_0'].unique())

print(f"\n📈 ความครอบคลุมข้อมูล:")
print(f"  - หลุมที่มี Lithology: {holes_with_lithology}/{total_holes} ({holes_with_lithology/total_holes*100:.1f}%)")
print(f"  - หลุมที่มี Samples: {holes_with_samples}/{total_holes} ({holes_with_samples/total_holes*100:.1f}%)")

# ตรวจสอบข้อมูลที่ซ้ำซ้อน
print(f"\n🔄 ข้อมูลซ้ำ:")
print(f"  - Collars ซ้ำ: {collars.duplicated().sum()}")
print(f"  - Samples ซ้ำ: {samples.duplicated().sum()}")
print(f"  - Lithology ซ้ำ: {lithology.duplicated().sum()}")


In [ ]:
# บันทึกข้อมูลเป็น CSV
print("💾 บันทึกข้อมูลเป็น CSV:")
print("=" * 40)

# สร้างโฟลเดอร์ output
import os
output_dir = "tutorial_output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"📁 สร้างโฟลเดอร์: {output_dir}")

# บันทึกข้อมูลแต่ละประเภท
try:
    collars.write_csv(f"{output_dir}/collars.csv")
    print(f"✅ Collars: {len(collars)} หลุม")
    
    samples.write_csv(f"{output_dir}/samples.csv")
    print(f"✅ Samples: {len(samples)} ตัวอย่าง")
    
    lithology.write_csv(f"{output_dir}/lithology.csv")
    print(f"✅ Lithology: {len(lithology)} ช่วง")
    
    print(f"\n🎉 บันทึกไฟล์ CSV สำเร็จในโฟลเดอร์: {output_dir}")
    
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")


## 🎯 สรุปการทำความสะอาดข้อมูล Hongsa Drilling Database

### ✅ สิ่งที่ได้ทำ:
1. **อ่านข้อมูลจาก Excel** - อ่าน DH70.xlsx worksheet DAT201
2. **ทำความสะอาดข้อมูล** - แปลง -1 เป็น NULL, ลบสูตร Excel
3. **แยกข้อมูลตามประเภท** - Collars, Samples, Lithology
4. **ตรวจสอบคุณภาพข้อมูล** - ข้อมูลซ้ำ, ข้อมูลหายไป
5. **บันทึกเป็น CSV** - สำหรับการใช้งานต่อไป

### 📊 ผลลัพธ์ที่คาดหวัง:
- **Collars**: ~70 หลุม (ข้อมูลปากหลุมเจาะ)
- **Samples**: ~8,500+ ตัวอย่าง (ผลการวิเคราะห์)
- **Lithology**: ~6,500+ ช่วง (ข้อมูลชั้นหิน)

### 🔄 ขั้นตอนต่อไป:
1. ใช้ `clean_and_create_db.py` เพื่อสร้างฐานข้อมูล SQLite
2. ใช้ `export_sqlite_to_csv.py` เพื่อส่งออกเป็น CSV สำหรับ SQL Server
3. ใช้ `sample_sql_queries.sql` สำหรับการสอบถามข้อมูล


In [ ]:
# Import libraries ที่จำเป็น
import polars as pl
import numpy as np
# ไม่ต้องใช้ pandas เพราะใช้ Polars แทน
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ตั้งค่า Polars เพื่อแสดงผลได้ดีขึ้น
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(10)

print("✅ Libraries โหลดสำเร็จแล้ว!")
print(f"Polars version: {pl.__version__}")
print(f"NumPy version: {np.__version__}")


In [ ]:
# สร้างข้อมูลตัวอย่างที่มีปัญหาต่างๆ
np.random.seed(42)

# สร้างข้อมูลพื้นฐาน
n_samples = 1000
data = {
    'id': range(1, n_samples + 1),
    'name': [f'Customer_{i:04d}' for i in range(1, n_samples + 1)],
    'age': np.random.normal(35, 10, n_samples).astype(int),
    'salary': np.random.normal(50000, 15000, n_samples),
    'email': [f'user{i}@example.com' for i in range(1, n_samples + 1)],
    'phone': [f'08{i%10000000000:010d}' for i in range(1, n_samples + 1)],
    'join_date': [(datetime.now() - timedelta(days=np.random.randint(0, 1000))).strftime('%Y-%m-%d') for _ in range(n_samples)],
    'department': np.random.choice(['IT', 'HR', 'Finance', 'Marketing', 'Sales'], n_samples),
    'status': np.random.choice(['Active', 'Inactive', 'Pending'], n_samples),
    'score': np.random.uniform(0, 100, n_samples)
}

# สร้าง Polars DataFrame
df_original = pl.DataFrame(data)

print("📊 ข้อมูลต้นฉบับ:")
print(f"ขนาดข้อมูล: {df_original.shape}")
print(f"คอลัมน์: {df_original.columns}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
df_original.head()


In [ ]:
# เพิ่มปัญหาต่างๆ ให้กับข้อมูล
df_dirty = df_original.clone()

# 1. เพิ่ม Missing Values (ข้อมูลหายไป)
missing_indices = np.random.choice(n_samples, size=int(0.1 * n_samples), replace=False)
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()).is_in(missing_indices))
    .then(None)
    .otherwise(pl.col('age'))
    .alias('age')
)

# 2. เพิ่มข้อมูลที่ผิดปกติ (Outliers)
outlier_indices = np.random.choice(n_samples, size=int(0.05 * n_samples), replace=False)
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()).is_in(outlier_indices))
    .then(pl.col('salary') * 10)  # เงินเดือนผิดปกติ
    .otherwise(pl.col('salary'))
    .alias('salary')
)

# 3. เพิ่มข้อมูลซ้ำ
duplicate_indices = np.random.choice(n_samples, size=int(0.03 * n_samples), replace=False)
duplicates = df_dirty[duplicate_indices]
df_dirty = pl.concat([df_dirty, duplicates])

# 4. เพิ่มข้อมูลที่รูปแบบไม่สม่ำเสมอ
inconsistent_indices = np.random.choice(len(df_dirty), size=int(0.08 * len(df_dirty)), replace=False)
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()).is_in(inconsistent_indices))
    .then(pl.lit('invalid_email'))
    .otherwise(pl.col('email'))
    .alias('email')
)

# 5. เพิ่มข้อมูลประเภทที่ไม่ถูกต้อง
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()) % 20 == 0)
    .then(pl.lit('invalid_date'))
    .otherwise(pl.col('join_date'))
    .alias('join_date')
)

print("🔧 ข้อมูลที่มีปัญหาต่างๆ:")
print(f"ขนาดข้อมูลหลังเพิ่มปัญหา: {df_dirty.shape}")
print(f"จำนวนข้อมูลซ้ำ: {df_dirty.shape[0] - df_original.shape[0]}")
print("\nตัวอย่างข้อมูลที่มีปัญหา:")
df_dirty.head(10)


## 1️⃣ การเตรียมข้อมูล (Data Preparation)

### การสร้างข้อมูลตัวอย่างสำหรับการทดสอบ

เราจะสร้างข้อมูลตัวอย่างที่มีปัญหาต่างๆ ที่พบได้บ่อยในการทำงานจริง:
- ข้อมูลที่หายไป (Missing Values)
- ข้อมูลที่ผิดปกติ (Outliers)
- ข้อมูลประเภทที่ไม่ถูกต้อง (Wrong Data Types)
- ข้อมูลที่ซ้ำกัน (Duplicates)
- ข้อมูลที่มีรูปแบบไม่สม่ำเสมอ (Inconsistent Format)


In [ ]:
# ตรวจสอบข้อมูลสถิติพื้นฐาน
print("📈 ข้อมูลสถิติพื้นฐาน:")
print("=" * 50)
print(df_dirty.describe())

print("\n📋 ข้อมูลประเภทคอลัมน์:")
print("=" * 50)
print(df_dirty.dtypes)

print("\n🔍 ข้อมูลที่หายไป (Missing Values):")
print("=" * 50)
missing_data = df_dirty.null_count()
print(missing_data)

print("\n📊 เปอร์เซ็นต์ข้อมูลที่หายไป:")
print("=" * 50)
missing_percentage = (missing_data / len(df_dirty) * 100).round(2)
for col in missing_percentage.columns:
    print(f"{col}: {missing_percentage[col][0]}%")


In [ ]:
# ตรวจสอบข้อมูลซ้ำ
print("🔄 ตรวจสอบข้อมูลซ้ำ:")
print("=" * 50)
duplicates = df_dirty.duplicated()
print(f"จำนวนแถวที่ซ้ำ: {duplicates.sum()}")

# ตรวจสอบข้อมูลซ้ำตามคอลัมน์เฉพาะ
duplicates_by_id = df_dirty.duplicated(subset=['id'])
print(f"จำนวนแถวที่ซ้ำตาม ID: {duplicates_by_id.sum()}")

print("\n📋 ตัวอย่างข้อมูลซ้ำ:")
print("=" * 50)
if duplicates.sum() > 0:
    duplicate_rows = df_dirty.filter(duplicates)
    print(duplicate_rows.head())

# ตรวจสอบข้อมูลที่ผิดปกติ (Outliers) ในคอลัมน์ตัวเลข
print("\n📊 ตรวจสอบข้อมูลที่ผิดปกติ (Outliers):")
print("=" * 50)
numeric_cols = ['age', 'salary', 'score']
for col in numeric_cols:
    if col in df_dirty.columns:
        Q1 = df_dirty[col].quantile(0.25)
        Q3 = df_dirty[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df_dirty.filter(
            (pl.col(col) < lower_bound) | (pl.col(col) > upper_bound)
        )
        print(f"{col}: {len(outliers)} outliers (ขอบเขต: {lower_bound:.2f} - {upper_bound:.2f})")


In [ ]:
# สร้างข้อมูลตัวอย่างที่มีปัญหาต่างๆ
np.random.seed(42)

# สร้างข้อมูลพื้นฐาน
n_samples = 1000
data = {
    'id': range(1, n_samples + 1),
    'name': [f'Customer_{i:04d}' for i in range(1, n_samples + 1)],
    'age': np.random.normal(35, 10, n_samples).astype(int),
    'salary': np.random.normal(50000, 15000, n_samples),
    'email': [f'user{i}@example.com' for i in range(1, n_samples + 1)],
    'phone': [f'08{i%10000000000:010d}' for i in range(1, n_samples + 1)],
    'join_date': [(datetime.now() - timedelta(days=np.random.randint(0, 1000))).strftime('%Y-%m-%d') for _ in range(n_samples)],
    'department': np.random.choice(['IT', 'HR', 'Finance', 'Marketing', 'Sales'], n_samples),
    'status': np.random.choice(['Active', 'Inactive', 'Pending'], n_samples),
    'score': np.random.uniform(0, 100, n_samples)
}

# สร้าง Polars DataFrame
df_original = pl.DataFrame(data)

print("📊 ข้อมูลต้นฉบับ:")
print(f"ขนาดข้อมูล: {df_original.shape}")
print(f"คอลัมน์: {df_original.columns}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
df_original.head()


In [ ]:
# เริ่มต้นการทำความสะอาดข้อมูล
df_cleaned = df_dirty.clone()

print("🧹 เริ่มต้นการทำความสะอาดข้อมูล...")
print(f"ขนาดข้อมูลก่อนทำความสะอาด: {df_cleaned.shape}")

# วิธีที่ 1: การลบข้อมูลที่หายไปทั้งหมด
print("\n1️⃣ วิธีที่ 1: การลบข้อมูลที่หายไปทั้งหมด")
df_drop_all = df_cleaned.drop_nulls()
print(f"ขนาดข้อมูลหลังลบข้อมูลที่หายไปทั้งหมด: {df_drop_all.shape}")
print(f"จำนวนแถวที่ถูกลบ: {df_cleaned.shape[0] - df_drop_all.shape[0]}")

# วิธีที่ 2: การลบเฉพาะแถวที่มีข้อมูลหายไปในคอลัมน์สำคัญ
print("\n2️⃣ วิธีที่ 2: การลบเฉพาะแถวที่มีข้อมูลหายไปในคอลัมน์สำคัญ")
df_drop_important = df_cleaned.drop_nulls(subset=['id', 'name', 'email'])
print(f"ขนาดข้อมูลหลังลบข้อมูลที่หายไปในคอลัมน์สำคัญ: {df_drop_important.shape}")

# วิธีที่ 3: การแทนที่ด้วยค่าเฉลี่ย (สำหรับข้อมูลตัวเลข)
print("\n3️⃣ วิธีที่ 3: การแทนที่ด้วยค่าเฉลี่ย")
df_fill_mean = df_cleaned.with_columns(
    pl.col('age').fill_null(pl.col('age').mean())
)
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ age ก่อน: {df_cleaned['age'].null_count()}")
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ age หลัง: {df_fill_mean['age'].null_count()}")

# วิธีที่ 4: การแทนที่ด้วยค่ากลาง (Median)
print("\n4️⃣ วิธีที่ 4: การแทนที่ด้วยค่ากลาง (Median)")
df_fill_median = df_cleaned.with_columns(
    pl.col('age').fill_null(pl.col('age').median())
)
print(f"ค่าเฉลี่ยของ age: {df_cleaned['age'].mean():.2f}")
print(f"ค่ากลางของ age: {df_cleaned['age'].median():.2f}")

# วิธีที่ 5: การแทนที่ด้วยค่าที่สมเหตุสมผล
print("\n5️⃣ วิธีที่ 5: การแทนที่ด้วยค่าที่สมเหตุสมผล")
df_fill_reasonable = df_cleaned.with_columns(
    pl.col('age').fill_null(30),  # ใช้ค่า 30 เป็นค่าเริ่มต้น
    pl.col('department').fill_null('Unknown')  # ใช้ 'Unknown' สำหรับ department
)
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ age หลัง: {df_fill_reasonable['age'].null_count()}")
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ department หลัง: {df_fill_reasonable['department'].null_count()}")


In [ ]:
# เพิ่มปัญหาต่างๆ ให้กับข้อมูล
df_dirty = df_original.clone()

# 1. เพิ่ม Missing Values (ข้อมูลหายไป)
missing_indices = np.random.choice(n_samples, size=int(0.1 * n_samples), replace=False)
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()).is_in(missing_indices))
    .then(None)
    .otherwise(pl.col('age'))
    .alias('age')
)

# 2. เพิ่มข้อมูลที่ผิดปกติ (Outliers)
outlier_indices = np.random.choice(n_samples, size=int(0.05 * n_samples), replace=False)
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()).is_in(outlier_indices))
    .then(pl.col('salary') * 10)  # เงินเดือนผิดปกติ
    .otherwise(pl.col('salary'))
    .alias('salary')
)

# 3. เพิ่มข้อมูลซ้ำ
duplicate_indices = np.random.choice(n_samples, size=int(0.03 * n_samples), replace=False)
duplicates = df_dirty[duplicate_indices]
df_dirty = pl.concat([df_dirty, duplicates])

# 4. เพิ่มข้อมูลที่รูปแบบไม่สม่ำเสมอ
inconsistent_indices = np.random.choice(len(df_dirty), size=int(0.08 * len(df_dirty)), replace=False)
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()).is_in(inconsistent_indices))
    .then(pl.lit('invalid_email'))
    .otherwise(pl.col('email'))
    .alias('email')
)

# 5. เพิ่มข้อมูลประเภทที่ไม่ถูกต้อง
df_dirty = df_dirty.with_columns(
    pl.when(pl.int_range(pl.len()) % 20 == 0)
    .then(pl.lit('invalid_date'))
    .otherwise(pl.col('join_date'))
    .alias('join_date')
)

print("🔧 ข้อมูลที่มีปัญหาต่างๆ:")
print(f"ขนาดข้อมูลหลังเพิ่มปัญหา: {df_dirty.shape}")
print(f"จำนวนข้อมูลซ้ำ: {df_dirty.shape[0] - df_original.shape[0]}")
print("\nตัวอย่างข้อมูลที่มีปัญหา:")
df_dirty.head(10)


In [ ]:
# การจัดการข้อมูลซ้ำ
print("🔄 การจัดการข้อมูลซ้ำ:")
print("=" * 50)

# ตรวจสอบข้อมูลซ้ำก่อนทำความสะอาด
duplicates_before = df_cleaned.duplicated().sum()
print(f"จำนวนข้อมูลซ้ำก่อนทำความสะอาด: {duplicates_before}")

# วิธีที่ 1: ลบข้อมูลซ้ำทั้งหมด (เก็บแถวแรก)
print("\n1️⃣ วิธีที่ 1: ลบข้อมูลซ้ำทั้งหมด (เก็บแถวแรก)")
df_no_duplicates = df_cleaned.unique()
print(f"ขนาดข้อมูลหลังลบข้อมูลซ้ำ: {df_no_duplicates.shape}")
print(f"จำนวนแถวที่ถูกลบ: {df_cleaned.shape[0] - df_no_duplicates.shape[0]}")

# วิธีที่ 2: ลบข้อมูลซ้ำตามคอลัมน์เฉพาะ (เช่น ID)
print("\n2️⃣ วิธีที่ 2: ลบข้อมูลซ้ำตามคอลัมน์ ID")
df_no_duplicates_id = df_cleaned.unique(subset=['id'])
print(f"ขนาดข้อมูลหลังลบข้อมูลซ้ำตาม ID: {df_no_duplicates_id.shape}")

# วิธีที่ 3: ลบข้อมูลซ้ำตามหลายคอลัมน์
print("\n3️⃣ วิธีที่ 3: ลบข้อมูลซ้ำตามหลายคอลัมน์")
df_no_duplicates_multi = df_cleaned.unique(subset=['id', 'email'])
print(f"ขนาดข้อมูลหลังลบข้อมูลซ้ำตาม ID และ Email: {df_no_duplicates_multi.shape}")

# วิธีที่ 4: ใช้ keep='last' เพื่อเก็บแถวสุดท้าย
print("\n4️⃣ วิธีที่ 4: เก็บแถวสุดท้ายของข้อมูลซ้ำ")
df_keep_last = df_cleaned.unique(subset=['id'], keep='last')
print(f"ขนาดข้อมูลหลังเก็บแถวสุดท้าย: {df_keep_last.shape}")

# ตรวจสอบข้อมูลซ้ำหลังทำความสะอาด
duplicates_after = df_no_duplicates.duplicated().sum()
print(f"\nจำนวนข้อมูลซ้ำหลังทำความสะอาด: {duplicates_after}")

# ใช้ข้อมูลที่ทำความสะอาดแล้วสำหรับขั้นตอนต่อไป
df_cleaned = df_no_duplicates


## 2️⃣ การตรวจสอบข้อมูล (Data Inspection)

### การสำรวจข้อมูลเพื่อหาปัญหา

ก่อนที่จะทำความสะอาดข้อมูล เราต้องเข้าใจปัญหาที่มีอยู่ก่อน โดยการตรวจสอบ:
- ข้อมูลสถิติพื้นฐาน (Basic Statistics)
- ข้อมูลที่หายไป (Missing Values)
- ข้อมูลที่ซ้ำกัน (Duplicates)
- ประเภทข้อมูล (Data Types)
- ข้อมูลที่ผิดปกติ (Outliers)


In [ ]:
# การจัดการข้อมูลที่ผิดปกติ
print("📊 การจัดการข้อมูลที่ผิดปกติ:")
print("=" * 50)

# ตรวจสอบข้อมูลที่ผิดปกติในคอลัมน์ salary
print("ตรวจสอบข้อมูลที่ผิดปกติในคอลัมน์ salary:")
print(f"ค่าเฉลี่ย: {df_cleaned['salary'].mean():.2f}")
print(f"ค่ากลาง: {df_cleaned['salary'].median():.2f}")
print(f"ค่าเบี่ยงเบนมาตรฐาน: {df_cleaned['salary'].std():.2f}")

# คำนวณขอบเขตสำหรับการตรวจจับข้อมูลที่ผิดปกติ
Q1 = df_cleaned['salary'].quantile(0.25)
Q3 = df_cleaned['salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nขอบเขตสำหรับการตรวจจับข้อมูลที่ผิดปกติ:")
print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"ขอบเขตล่าง: {lower_bound:.2f}")
print(f"ขอบเขตบน: {upper_bound:.2f}")

# ตรวจสอบข้อมูลที่ผิดปกติ
outliers = df_cleaned.filter(
    (pl.col('salary') < lower_bound) | (pl.col('salary') > upper_bound)
)
print(f"\nจำนวนข้อมูลที่ผิดปกติ: {len(outliers)}")

# วิธีที่ 1: ลบข้อมูลที่ผิดปกติ
print("\n1️⃣ วิธีที่ 1: ลบข้อมูลที่ผิดปกติ")
df_no_outliers = df_cleaned.filter(
    (pl.col('salary') >= lower_bound) & (pl.col('salary') <= upper_bound)
)
print(f"ขนาดข้อมูลหลังลบข้อมูลที่ผิดปกติ: {df_no_outliers.shape}")
print(f"จำนวนแถวที่ถูกลบ: {df_cleaned.shape[0] - df_no_outliers.shape[0]}")

# วิธีที่ 2: แทนที่ด้วยค่าขอบเขต
print("\n2️⃣ วิธีที่ 2: แทนที่ด้วยค่าขอบเขต")
df_capped = df_cleaned.with_columns(
    pl.col('salary').clip(lower_bound, upper_bound).alias('salary')
)
print(f"ค่าเฉลี่ยหลังการแทนที่: {df_capped['salary'].mean():.2f}")

# วิธีที่ 3: แทนที่ด้วยค่าเฉลี่ย
print("\n3️⃣ วิธีที่ 3: แทนที่ด้วยค่าเฉลี่ย")
mean_salary = df_cleaned['salary'].mean()
df_mean_replaced = df_cleaned.with_columns(
    pl.when((pl.col('salary') < lower_bound) | (pl.col('salary') > upper_bound))
    .then(mean_salary)
    .otherwise(pl.col('salary'))
    .alias('salary')
)
print(f"ค่าเฉลี่ยหลังการแทนที่ด้วยค่าเฉลี่ย: {df_mean_replaced['salary'].mean():.2f}")

# วิธีที่ 4: ใช้การแปลงข้อมูล (Log transformation)
print("\n4️⃣ วิธีที่ 4: ใช้การแปลงข้อมูล (Log transformation)")
df_log_transformed = df_cleaned.with_columns(
    pl.col('salary').log().alias('salary_log')
)
print(f"ค่าเฉลี่ยของ salary_log: {df_log_transformed['salary_log'].mean():.2f}")

# ใช้ข้อมูลที่ทำความสะอาดแล้วสำหรับขั้นตอนต่อไป
df_cleaned = df_capped


In [ ]:
# ตรวจสอบข้อมูลสถิติพื้นฐาน
print("📈 ข้อมูลสถิติพื้นฐาน:")
print("=" * 50)
print(df_dirty.describe())

print("\n📋 ข้อมูลประเภทคอลัมน์:")
print("=" * 50)
print(df_dirty.dtypes)

print("\n🔍 ข้อมูลที่หายไป (Missing Values):")
print("=" * 50)
missing_data = df_dirty.null_count()
print(missing_data)

print("\n📊 เปอร์เซ็นต์ข้อมูลที่หายไป:")
print("=" * 50)
missing_percentage = (missing_data / len(df_dirty) * 100).round(2)
for col in missing_percentage.columns:
    print(f"{col}: {missing_percentage[col][0]}%")


In [ ]:
# การแปลงข้อมูล
print("🔄 การแปลงข้อมูล:")
print("=" * 50)

# ตรวจสอบประเภทข้อมูลปัจจุบัน
print("ประเภทข้อมูลปัจจุบัน:")
print(df_cleaned.dtypes)

# 1. การแปลงประเภทข้อมูล
print("\n1️⃣ การแปลงประเภทข้อมูล")
df_transformed = df_cleaned.with_columns([
    pl.col('id').cast(pl.Utf8),  # แปลง ID เป็น string
    pl.col('age').cast(pl.Int32),  # แปลง age เป็น int32
    pl.col('salary').cast(pl.Float64),  # แปลง salary เป็น float64
    pl.col('join_date').str.strptime(pl.Date, '%Y-%m-%d'),  # แปลง join_date เป็น Date
    pl.col('department').cast(pl.Categorical),  # แปลง department เป็น Categorical
    pl.col('status').cast(pl.Categorical),  # แปลง status เป็น Categorical
])

print("ประเภทข้อมูลหลังการแปลง:")
print(df_transformed.dtypes)

# 2. การแปลงรูปแบบข้อมูล
print("\n2️⃣ การแปลงรูปแบบข้อมูล")
df_formatted = df_transformed.with_columns([
    # แปลงชื่อให้เป็นตัวพิมพ์ใหญ่
    pl.col('name').str.to_uppercase().alias('name'),
    # แปลง email ให้เป็นตัวพิมพ์เล็ก
    pl.col('email').str.to_lowercase().alias('email'),
    # แปลงเบอร์โทรศัพท์ให้มีรูปแบบที่สม่ำเสมอ
    pl.col('phone').str.replace_all(r'(\d{3})(\d{3})(\d{4})', r'$1-$2-$3').alias('phone'),
])

print("ตัวอย่างข้อมูลหลังการแปลงรูปแบบ:")
print(df_formatted.select(['name', 'email', 'phone']).head())

# 3. การสร้างคอลัมน์ใหม่
print("\n3️⃣ การสร้างคอลัมน์ใหม่")
df_with_new_cols = df_formatted.with_columns([
    # สร้างคอลัมน์ salary_category
    pl.when(pl.col('salary') < 30000)
    .then(pl.lit('Low'))
    .when(pl.col('salary') < 60000)
    .then(pl.lit('Medium'))
    .otherwise(pl.lit('High'))
    .alias('salary_category'),
    
    # สร้างคอลัมน์ age_group
    pl.when(pl.col('age') < 25)
    .then(pl.lit('Young'))
    .when(pl.col('age') < 50)
    .then(pl.lit('Middle-aged'))
    .otherwise(pl.lit('Senior'))
    .alias('age_group'),
    
    # สร้างคอลัมน์ years_of_service
    (pl.lit(datetime.now().date()) - pl.col('join_date')).dt.total_days() / 365.25
    .round(1)
    .alias('years_of_service'),
    
    # สร้างคอลัมน์ is_active
    pl.col('status').eq('Active').alias('is_active')
])

print("ตัวอย่างข้อมูลพร้อมคอลัมน์ใหม่:")
print(df_with_new_cols.select(['name', 'salary_category', 'age_group', 'years_of_service', 'is_active']).head())

# 4. การจัดกลุ่มข้อมูล
print("\n4️⃣ การจัดกลุ่มข้อมูล")
grouped_data = df_with_new_cols.group_by(['department', 'salary_category']).agg([
    pl.count().alias('count'),
    pl.col('salary').mean().round(2).alias('avg_salary'),
    pl.col('age').mean().round(1).alias('avg_age'),
    pl.col('score').mean().round(2).alias('avg_score')
]).sort('department', 'salary_category')

print("ข้อมูลที่จัดกลุ่มตาม department และ salary_category:")
print(grouped_data)


In [ ]:
# ตรวจสอบข้อมูลซ้ำ
print("🔄 ตรวจสอบข้อมูลซ้ำ:")
print("=" * 50)
duplicates = df_dirty.duplicated()
print(f"จำนวนแถวที่ซ้ำ: {duplicates.sum()}")

# ตรวจสอบข้อมูลซ้ำตามคอลัมน์เฉพาะ
duplicates_by_id = df_dirty.duplicated(subset=['id'])
print(f"จำนวนแถวที่ซ้ำตาม ID: {duplicates_by_id.sum()}")

print("\n📋 ตัวอย่างข้อมูลซ้ำ:")
print("=" * 50)
if duplicates.sum() > 0:
    duplicate_rows = df_dirty.filter(duplicates)
    print(duplicate_rows.head())

# ตรวจสอบข้อมูลที่ผิดปกติ (Outliers) ในคอลัมน์ตัวเลข
print("\n📊 ตรวจสอบข้อมูลที่ผิดปกติ (Outliers):")
print("=" * 50)
numeric_cols = ['age', 'salary', 'score']
for col in numeric_cols:
    if col in df_dirty.columns:
        Q1 = df_dirty[col].quantile(0.25)
        Q3 = df_dirty[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df_dirty.filter(
            (pl.col(col) < lower_bound) | (pl.col(col) > upper_bound)
        )
        print(f"{col}: {len(outliers)} outliers (ขอบเขต: {lower_bound:.2f} - {upper_bound:.2f})")


In [ ]:
# การตรวจสอบผลลัพธ์
print("✅ การตรวจสอบผลลัพธ์:")
print("=" * 50)

# 1. ตรวจสอบข้อมูลที่หายไป
print("1️⃣ ตรวจสอบข้อมูลที่หายไป:")
missing_after = df_with_new_cols.null_count()
print("จำนวนข้อมูลที่หายไปในแต่ละคอลัมน์:")
for col in missing_after.columns:
    count = missing_after[col][0]
    percentage = (count / len(df_with_new_cols) * 100)
    print(f"  {col}: {count} ({percentage:.2f}%)")

# 2. ตรวจสอบข้อมูลซ้ำ
print("\n2️⃣ ตรวจสอบข้อมูลซ้ำ:")
duplicates_after = df_with_new_cols.duplicated().sum()
print(f"จำนวนข้อมูลซ้ำ: {duplicates_after}")

# 3. ตรวจสอบข้อมูลที่ผิดปกติ
print("\n3️⃣ ตรวจสอบข้อมูลที่ผิดปกติ:")
Q1 = df_with_new_cols['salary'].quantile(0.25)
Q3 = df_with_new_cols['salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_after = df_with_new_cols.filter(
    (pl.col('salary') < lower_bound) | (pl.col('salary') > upper_bound)
)
print(f"จำนวนข้อมูลที่ผิดปกติใน salary: {len(outliers_after)}")

# 4. ตรวจสอบประเภทข้อมูล
print("\n4️⃣ ตรวจสอบประเภทข้อมูล:")
print("ประเภทข้อมูลปัจจุบัน:")
for col, dtype in zip(df_with_new_cols.columns, df_with_new_cols.dtypes):
    print(f"  {col}: {dtype}")

# 5. ตรวจสอบความสมบูรณ์ของข้อมูล
print("\n5️⃣ ตรวจสอบความสมบูรณ์ของข้อมูล:")
print(f"จำนวนแถวทั้งหมด: {len(df_with_new_cols)}")
print(f"จำนวนคอลัมน์ทั้งหมด: {len(df_with_new_cols.columns)}")

# ตรวจสอบข้อมูลที่สำคัญ
print("\nข้อมูลสถิติพื้นฐาน:")
print(df_with_new_cols.describe())

# ตรวจสอบข้อมูลที่ไม่มีค่าในคอลัมน์สำคัญ
critical_cols = ['id', 'name', 'email']
for col in critical_cols:
    if col in df_with_new_cols.columns:
        null_count = df_with_new_cols[col].null_count()
        print(f"คอลัมน์ {col}: {null_count} ค่าที่หายไป")

print("\n✅ การตรวจสอบเสร็จสิ้น!")


## 3️⃣ การจัดการ Missing Values

### วิธีการแก้ไขข้อมูลที่หายไป

ข้อมูลที่หายไป (Missing Values) เป็นปัญหาที่พบบ่อยมาก มีหลายวิธีในการจัดการ:

1. **การลบข้อมูล** - ลบแถวหรือคอลัมน์ที่มีข้อมูลหายไป
2. **การแทนที่ด้วยค่าเฉลี่ย/ค่ากลาง** - ใช้ค่าสถิติแทนที่
3. **การแทนที่ด้วยค่าที่สมเหตุสมผล** - ใช้ค่าที่เหมาะสมกับบริบท
4. **การแทนที่ด้วยค่าก่อนหน้า/ถัดไป** - ใช้ค่าจากแถวใกล้เคียง


In [ ]:
# การแสดงผลและ Visualization
print("📊 การแสดงผลและ Visualization:")
print("=" * 50)

# 1. แสดงผลข้อมูลพื้นฐาน
print("1️⃣ ข้อมูลพื้นฐาน:")
print("ข้อมูลต้นฉบับ (5 แถวแรก):")
print(df_dirty.head())
print("\nข้อมูลหลังทำความสะอาด (5 แถวแรก):")
print(df_with_new_cols.head())

# 2. แสดงผลสถิติ
print("\n2️⃣ สถิติพื้นฐาน:")
print("ข้อมูลต้นฉบับ:")
print(df_dirty.describe())
print("\nข้อมูลหลังทำความสะอาด:")
print(df_with_new_cols.describe())

# 3. แสดงผลการกระจายข้อมูล
print("\n3️⃣ การกระจายข้อมูล:")
print("การกระจายของ salary:")
print(f"  ค่าเฉลี่ย: {df_with_new_cols['salary'].mean():.2f}")
print(f"  ค่ากลาง: {df_with_new_cols['salary'].median():.2f}")
print(f"  ค่าเบี่ยงเบนมาตรฐาน: {df_with_new_cols['salary'].std():.2f}")
print(f"  ค่าต่ำสุด: {df_with_new_cols['salary'].min():.2f}")
print(f"  ค่าสูงสุด: {df_with_new_cols['salary'].max():.2f}")

print("\nการกระจายของ age:")
print(f"  ค่าเฉลี่ย: {df_with_new_cols['age'].mean():.2f}")
print(f"  ค่ากลาง: {df_with_new_cols['age'].median():.2f}")
print(f"  ค่าเบี่ยงเบนมาตรฐาน: {df_with_new_cols['age'].std():.2f}")

# 4. แสดงผลการเปรียบเทียบ
print("\n4️⃣ การเปรียบเทียบข้อมูลก่อนและหลังทำความสะอาด:")
print("ขนาดข้อมูล:")
print(f"  ก่อนทำความสะอาด: {df_dirty.shape}")
print(f"  หลังทำความสะอาด: {df_with_new_cols.shape}")

print("\nข้อมูลที่หายไป:")
print("  ก่อนทำความสะอาด:")
for col in df_dirty.columns:
    null_count = df_dirty[col].null_count()
    print(f"    {col}: {null_count}")
print("  หลังทำความสะอาด:")
for col in df_with_new_cols.columns:
    null_count = df_with_new_cols[col].null_count()
    print(f"    {col}: {null_count}")

print("\nข้อมูลซ้ำ:")
print(f"  ก่อนทำความสะอาด: {df_dirty.duplicated().sum()}")
print(f"  หลังทำความสะอาด: {df_with_new_cols.duplicated().sum()}")

# 5. แสดงผลข้อมูลที่จัดกลุ่ม
print("\n5️⃣ ข้อมูลที่จัดกลุ่ม:")
print("ตาม department:")
dept_summary = df_with_new_cols.group_by('department').agg([
    pl.count().alias('count'),
    pl.col('salary').mean().round(2).alias('avg_salary'),
    pl.col('age').mean().round(1).alias('avg_age')
]).sort('department')
print(dept_summary)

print("\nตาม salary_category:")
salary_summary = df_with_new_cols.group_by('salary_category').agg([
    pl.count().alias('count'),
    pl.col('salary').mean().round(2).alias('avg_salary'),
    pl.col('age').mean().round(1).alias('avg_age')
]).sort('salary_category')
print(salary_summary)


In [ ]:
# เริ่มต้นการทำความสะอาดข้อมูล
df_cleaned = df_dirty.clone()

print("🧹 เริ่มต้นการทำความสะอาดข้อมูล...")
print(f"ขนาดข้อมูลก่อนทำความสะอาด: {df_cleaned.shape}")

# วิธีที่ 1: การลบข้อมูลที่หายไปทั้งหมด
print("\n1️⃣ วิธีที่ 1: การลบข้อมูลที่หายไปทั้งหมด")
df_drop_all = df_cleaned.drop_nulls()
print(f"ขนาดข้อมูลหลังลบข้อมูลที่หายไปทั้งหมด: {df_drop_all.shape}")
print(f"จำนวนแถวที่ถูกลบ: {df_cleaned.shape[0] - df_drop_all.shape[0]}")

# วิธีที่ 2: การลบเฉพาะแถวที่มีข้อมูลหายไปในคอลัมน์สำคัญ
print("\n2️⃣ วิธีที่ 2: การลบเฉพาะแถวที่มีข้อมูลหายไปในคอลัมน์สำคัญ")
df_drop_important = df_cleaned.drop_nulls(subset=['id', 'name', 'email'])
print(f"ขนาดข้อมูลหลังลบข้อมูลที่หายไปในคอลัมน์สำคัญ: {df_drop_important.shape}")

# วิธีที่ 3: การแทนที่ด้วยค่าเฉลี่ย (สำหรับข้อมูลตัวเลข)
print("\n3️⃣ วิธีที่ 3: การแทนที่ด้วยค่าเฉลี่ย")
df_fill_mean = df_cleaned.with_columns(
    pl.col('age').fill_null(pl.col('age').mean())
)
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ age ก่อน: {df_cleaned['age'].null_count()}")
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ age หลัง: {df_fill_mean['age'].null_count()}")

# วิธีที่ 4: การแทนที่ด้วยค่ากลาง (Median)
print("\n4️⃣ วิธีที่ 4: การแทนที่ด้วยค่ากลาง (Median)")
df_fill_median = df_cleaned.with_columns(
    pl.col('age').fill_null(pl.col('age').median())
)
print(f"ค่าเฉลี่ยของ age: {df_cleaned['age'].mean():.2f}")
print(f"ค่ากลางของ age: {df_cleaned['age'].median():.2f}")

# วิธีที่ 5: การแทนที่ด้วยค่าที่สมเหตุสมผล
print("\n5️⃣ วิธีที่ 5: การแทนที่ด้วยค่าที่สมเหตุสมผล")
df_fill_reasonable = df_cleaned.with_columns(
    pl.col('age').fill_null(30),  # ใช้ค่า 30 เป็นค่าเริ่มต้น
    pl.col('department').fill_null('Unknown')  # ใช้ 'Unknown' สำหรับ department
)
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ age หลัง: {df_fill_reasonable['age'].null_count()}")
print(f"จำนวนข้อมูลที่หายไปในคอลัมน์ department หลัง: {df_fill_reasonable['department'].null_count()}")


## 4️⃣ การจัดการข้อมูลซ้ำ (Duplicate Data)

### วิธีการจัดการข้อมูลที่ซ้ำกัน

ข้อมูลซ้ำเป็นปัญหาที่ทำให้การวิเคราะห์ผิดพลาด มีหลายวิธีในการจัดการ:

1. **การลบข้อมูลซ้ำทั้งหมด** - ลบแถวที่ซ้ำกันทั้งหมด
2. **การเก็บข้อมูลซ้ำแค่แถวแรก** - เก็บแถวแรก ลบแถวที่เหลือ
3. **การเก็บข้อมูลซ้ำแค่แถวสุดท้าย** - เก็บแถวสุดท้าย ลบแถวก่อนหน้า
4. **การลบข้อมูลซ้ำตามคอลัมน์เฉพาะ** - ใช้คอลัมน์สำคัญในการตัดสินใจ


In [ ]:
# การจัดการข้อมูลซ้ำ
print("🔄 การจัดการข้อมูลซ้ำ:")
print("=" * 50)

# ตรวจสอบข้อมูลซ้ำก่อนทำความสะอาด
duplicates_before = df_cleaned.duplicated().sum()
print(f"จำนวนข้อมูลซ้ำก่อนทำความสะอาด: {duplicates_before}")

# วิธีที่ 1: ลบข้อมูลซ้ำทั้งหมด (เก็บแถวแรก)
print("\n1️⃣ วิธีที่ 1: ลบข้อมูลซ้ำทั้งหมด (เก็บแถวแรก)")
df_no_duplicates = df_cleaned.unique()
print(f"ขนาดข้อมูลหลังลบข้อมูลซ้ำ: {df_no_duplicates.shape}")
print(f"จำนวนแถวที่ถูกลบ: {df_cleaned.shape[0] - df_no_duplicates.shape[0]}")

# วิธีที่ 2: ลบข้อมูลซ้ำตามคอลัมน์เฉพาะ (เช่น ID)
print("\n2️⃣ วิธีที่ 2: ลบข้อมูลซ้ำตามคอลัมน์ ID")
df_no_duplicates_id = df_cleaned.unique(subset=['id'])
print(f"ขนาดข้อมูลหลังลบข้อมูลซ้ำตาม ID: {df_no_duplicates_id.shape}")

# วิธีที่ 3: ลบข้อมูลซ้ำตามหลายคอลัมน์
print("\n3️⃣ วิธีที่ 3: ลบข้อมูลซ้ำตามหลายคอลัมน์")
df_no_duplicates_multi = df_cleaned.unique(subset=['id', 'email'])
print(f"ขนาดข้อมูลหลังลบข้อมูลซ้ำตาม ID และ Email: {df_no_duplicates_multi.shape}")

# วิธีที่ 4: ใช้ keep='last' เพื่อเก็บแถวสุดท้าย
print("\n4️⃣ วิธีที่ 4: เก็บแถวสุดท้ายของข้อมูลซ้ำ")
df_keep_last = df_cleaned.unique(subset=['id'], keep='last')
print(f"ขนาดข้อมูลหลังเก็บแถวสุดท้าย: {df_keep_last.shape}")

# ตรวจสอบข้อมูลซ้ำหลังทำความสะอาด
duplicates_after = df_no_duplicates.duplicated().sum()
print(f"\nจำนวนข้อมูลซ้ำหลังทำความสะอาด: {duplicates_after}")

# ใช้ข้อมูลที่ทำความสะอาดแล้วสำหรับขั้นตอนต่อไป
df_cleaned = df_no_duplicates


## 5️⃣ การจัดการข้อมูลที่ผิดปกติ (Outliers)

### วิธีการจัดการข้อมูลที่ผิดปกติ

ข้อมูลที่ผิดปกติ (Outliers) เป็นข้อมูลที่แตกต่างจากข้อมูลส่วนใหญ่ มีหลายวิธีในการจัดการ:

1. **การลบข้อมูลที่ผิดปกติ** - ลบข้อมูลที่อยู่นอกขอบเขตที่กำหนด
2. **การแทนที่ด้วยค่าขอบเขต** - ใช้ค่าขอบเขตแทนที่ข้อมูลที่ผิดปกติ
3. **การแทนที่ด้วยค่าเฉลี่ย** - ใช้ค่าเฉลี่ยแทนที่ข้อมูลที่ผิดปกติ
4. **การแปลงข้อมูล** - ใช้การแปลงทางคณิตศาสตร์เพื่อลดผลกระทบ


In [ ]:
# การจัดการข้อมูลที่ผิดปกติ
print("📊 การจัดการข้อมูลที่ผิดปกติ:")
print("=" * 50)

# ตรวจสอบข้อมูลที่ผิดปกติในคอลัมน์ salary
print("ตรวจสอบข้อมูลที่ผิดปกติในคอลัมน์ salary:")
print(f"ค่าเฉลี่ย: {df_cleaned['salary'].mean():.2f}")
print(f"ค่ากลาง: {df_cleaned['salary'].median():.2f}")
print(f"ค่าเบี่ยงเบนมาตรฐาน: {df_cleaned['salary'].std():.2f}")

# คำนวณขอบเขตสำหรับการตรวจจับข้อมูลที่ผิดปกติ
Q1 = df_cleaned['salary'].quantile(0.25)
Q3 = df_cleaned['salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nขอบเขตสำหรับการตรวจจับข้อมูลที่ผิดปกติ:")
print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"ขอบเขตล่าง: {lower_bound:.2f}")
print(f"ขอบเขตบน: {upper_bound:.2f}")

# ตรวจสอบข้อมูลที่ผิดปกติ
outliers = df_cleaned.filter(
    (pl.col('salary') < lower_bound) | (pl.col('salary') > upper_bound)
)
print(f"\nจำนวนข้อมูลที่ผิดปกติ: {len(outliers)}")

# วิธีที่ 1: ลบข้อมูลที่ผิดปกติ
print("\n1️⃣ วิธีที่ 1: ลบข้อมูลที่ผิดปกติ")
df_no_outliers = df_cleaned.filter(
    (pl.col('salary') >= lower_bound) & (pl.col('salary') <= upper_bound)
)
print(f"ขนาดข้อมูลหลังลบข้อมูลที่ผิดปกติ: {df_no_outliers.shape}")
print(f"จำนวนแถวที่ถูกลบ: {df_cleaned.shape[0] - df_no_outliers.shape[0]}")

# วิธีที่ 2: แทนที่ด้วยค่าขอบเขต
print("\n2️⃣ วิธีที่ 2: แทนที่ด้วยค่าขอบเขต")
df_capped = df_cleaned.with_columns(
    pl.col('salary').clip(lower_bound, upper_bound).alias('salary')
)
print(f"ค่าเฉลี่ยหลังการแทนที่: {df_capped['salary'].mean():.2f}")

# วิธีที่ 3: แทนที่ด้วยค่าเฉลี่ย
print("\n3️⃣ วิธีที่ 3: แทนที่ด้วยค่าเฉลี่ย")
mean_salary = df_cleaned['salary'].mean()
df_mean_replaced = df_cleaned.with_columns(
    pl.when((pl.col('salary') < lower_bound) | (pl.col('salary') > upper_bound))
    .then(mean_salary)
    .otherwise(pl.col('salary'))
    .alias('salary')
)
print(f"ค่าเฉลี่ยหลังการแทนที่ด้วยค่าเฉลี่ย: {df_mean_replaced['salary'].mean():.2f}")

# วิธีที่ 4: ใช้การแปลงข้อมูล (Log transformation)
print("\n4️⃣ วิธีที่ 4: ใช้การแปลงข้อมูล (Log transformation)")
df_log_transformed = df_cleaned.with_columns(
    pl.col('salary').log().alias('salary_log')
)
print(f"ค่าเฉลี่ยของ salary_log: {df_log_transformed['salary_log'].mean():.2f}")

# ใช้ข้อมูลที่ทำความสะอาดแล้วสำหรับขั้นตอนต่อไป
df_cleaned = df_capped


## 6️⃣ การแปลงข้อมูล (Data Transformation)

### วิธีการแปลงข้อมูลให้เหมาะสม

การแปลงข้อมูลเป็นขั้นตอนสำคัญในการเตรียมข้อมูลให้พร้อมสำหรับการวิเคราะห์:

1. **การแปลงประเภทข้อมูล** - เปลี่ยนประเภทข้อมูลให้เหมาะสม
2. **การแปลงรูปแบบข้อมูล** - เปลี่ยนรูปแบบให้สม่ำเสมอ
3. **การสร้างคอลัมน์ใหม่** - สร้างคอลัมน์ที่ได้จากการคำนวณ
4. **การจัดกลุ่มข้อมูล** - จัดกลุ่มข้อมูลตามเงื่อนไข


In [ ]:
# การแปลงข้อมูล
print("🔄 การแปลงข้อมูล:")
print("=" * 50)

# ตรวจสอบประเภทข้อมูลปัจจุบัน
print("ประเภทข้อมูลปัจจุบัน:")
print(df_cleaned.dtypes)

# 1. การแปลงประเภทข้อมูล
print("\n1️⃣ การแปลงประเภทข้อมูล")
df_transformed = df_cleaned.with_columns([
    pl.col('id').cast(pl.Utf8),  # แปลง ID เป็น string
    pl.col('age').cast(pl.Int32),  # แปลง age เป็น int32
    pl.col('salary').cast(pl.Float64),  # แปลง salary เป็น float64
    pl.col('join_date').str.strptime(pl.Date, '%Y-%m-%d'),  # แปลง join_date เป็น Date
    pl.col('department').cast(pl.Categorical),  # แปลง department เป็น Categorical
    pl.col('status').cast(pl.Categorical),  # แปลง status เป็น Categorical
])

print("ประเภทข้อมูลหลังการแปลง:")
print(df_transformed.dtypes)

# 2. การแปลงรูปแบบข้อมูล
print("\n2️⃣ การแปลงรูปแบบข้อมูล")
df_formatted = df_transformed.with_columns([
    # แปลงชื่อให้เป็นตัวพิมพ์ใหญ่
    pl.col('name').str.to_uppercase().alias('name'),
    # แปลง email ให้เป็นตัวพิมพ์เล็ก
    pl.col('email').str.to_lowercase().alias('email'),
    # แปลงเบอร์โทรศัพท์ให้มีรูปแบบที่สม่ำเสมอ
    pl.col('phone').str.replace_all(r'(\d{3})(\d{3})(\d{4})', r'$1-$2-$3').alias('phone'),
])

print("ตัวอย่างข้อมูลหลังการแปลงรูปแบบ:")
print(df_formatted.select(['name', 'email', 'phone']).head())

# 3. การสร้างคอลัมน์ใหม่
print("\n3️⃣ การสร้างคอลัมน์ใหม่")
df_with_new_cols = df_formatted.with_columns([
    # สร้างคอลัมน์ salary_category
    pl.when(pl.col('salary') < 30000)
    .then(pl.lit('Low'))
    .when(pl.col('salary') < 60000)
    .then(pl.lit('Medium'))
    .otherwise(pl.lit('High'))
    .alias('salary_category'),
    
    # สร้างคอลัมน์ age_group
    pl.when(pl.col('age') < 25)
    .then(pl.lit('Young'))
    .when(pl.col('age') < 50)
    .then(pl.lit('Middle-aged'))
    .otherwise(pl.lit('Senior'))
    .alias('age_group'),
    
    # สร้างคอลัมน์ years_of_service
    (pl.lit(datetime.now().date()) - pl.col('join_date')).dt.total_days() / 365.25
    .round(1)
    .alias('years_of_service'),
    
    # สร้างคอลัมน์ is_active
    pl.col('status').eq('Active').alias('is_active')
])

print("ตัวอย่างข้อมูลพร้อมคอลัมน์ใหม่:")
print(df_with_new_cols.select(['name', 'salary_category', 'age_group', 'years_of_service', 'is_active']).head())

# 4. การจัดกลุ่มข้อมูล
print("\n4️⃣ การจัดกลุ่มข้อมูล")
grouped_data = df_with_new_cols.group_by(['department', 'salary_category']).agg([
    pl.count().alias('count'),
    pl.col('salary').mean().round(2).alias('avg_salary'),
    pl.col('age').mean().round(1).alias('avg_age'),
    pl.col('score').mean().round(2).alias('avg_score')
]).sort('department', 'salary_category')

print("ข้อมูลที่จัดกลุ่มตาม department และ salary_category:")
print(grouped_data)


## 7️⃣ การตรวจสอบผลลัพธ์ (Data Validation)

### การยืนยันความถูกต้องของข้อมูลหลังทำความสะอาด

หลังจากทำความสะอาดข้อมูลแล้ว ต้องตรวจสอบว่าข้อมูลมีความถูกต้องและพร้อมใช้งาน:

1. **การตรวจสอบข้อมูลที่หายไป** - ตรวจสอบว่าข้อมูลที่หายไปได้รับการจัดการแล้ว
2. **การตรวจสอบข้อมูลซ้ำ** - ตรวจสอบว่าข้อมูลซ้ำถูกลบแล้ว
3. **การตรวจสอบข้อมูลที่ผิดปกติ** - ตรวจสอบว่าข้อมูลที่ผิดปกติได้รับการแก้ไขแล้ว
4. **การตรวจสอบประเภทข้อมูล** - ตรวจสอบว่าประเภทข้อมูลถูกต้องแล้ว
5. **การตรวจสอบความสมบูรณ์** - ตรวจสอบว่าข้อมูลมีความสมบูรณ์และสอดคล้องกัน


In [ ]:
# การตรวจสอบผลลัพธ์
print("✅ การตรวจสอบผลลัพธ์:")
print("=" * 50)

# 1. ตรวจสอบข้อมูลที่หายไป
print("1️⃣ ตรวจสอบข้อมูลที่หายไป:")
missing_after = df_with_new_cols.null_count()
print("จำนวนข้อมูลที่หายไปในแต่ละคอลัมน์:")
for col in missing_after.columns:
    count = missing_after[col][0]
    percentage = (count / len(df_with_new_cols) * 100)
    print(f"  {col}: {count} ({percentage:.2f}%)")

# 2. ตรวจสอบข้อมูลซ้ำ
print("\n2️⃣ ตรวจสอบข้อมูลซ้ำ:")
duplicates_after = df_with_new_cols.duplicated().sum()
print(f"จำนวนข้อมูลซ้ำ: {duplicates_after}")

# 3. ตรวจสอบข้อมูลที่ผิดปกติ
print("\n3️⃣ ตรวจสอบข้อมูลที่ผิดปกติ:")
Q1 = df_with_new_cols['salary'].quantile(0.25)
Q3 = df_with_new_cols['salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_after = df_with_new_cols.filter(
    (pl.col('salary') < lower_bound) | (pl.col('salary') > upper_bound)
)
print(f"จำนวนข้อมูลที่ผิดปกติใน salary: {len(outliers_after)}")

# 4. ตรวจสอบประเภทข้อมูล
print("\n4️⃣ ตรวจสอบประเภทข้อมูล:")
print("ประเภทข้อมูลปัจจุบัน:")
for col, dtype in zip(df_with_new_cols.columns, df_with_new_cols.dtypes):
    print(f"  {col}: {dtype}")

# 5. ตรวจสอบความสมบูรณ์ของข้อมูล
print("\n5️⃣ ตรวจสอบความสมบูรณ์ของข้อมูล:")
print(f"จำนวนแถวทั้งหมด: {len(df_with_new_cols)}")
print(f"จำนวนคอลัมน์ทั้งหมด: {len(df_with_new_cols.columns)}")

# ตรวจสอบข้อมูลที่สำคัญ
print("\nข้อมูลสถิติพื้นฐาน:")
print(df_with_new_cols.describe())

# ตรวจสอบข้อมูลที่ไม่มีค่าในคอลัมน์สำคัญ
critical_cols = ['id', 'name', 'email']
for col in critical_cols:
    if col in df_with_new_cols.columns:
        null_count = df_with_new_cols[col].null_count()
        print(f"คอลัมน์ {col}: {null_count} ค่าที่หายไป")

print("\n✅ การตรวจสอบเสร็จสิ้น!")


## 8️⃣ การแสดงผลและ Visualization

### การแสดงผลข้อมูลเพื่อให้เห็นภาพชัดเจน

การแสดงผลข้อมูลเป็นส่วนสำคัญในการทำความเข้าใจข้อมูลและตรวจสอบผลลัพธ์:

1. **การแสดงผลข้อมูลพื้นฐาน** - แสดงข้อมูลในรูปแบบตาราง
2. **การแสดงผลสถิติ** - แสดงสถิติพื้นฐานของข้อมูล
3. **การแสดงผลการกระจาย** - แสดงการกระจายของข้อมูล
4. **การแสดงผลการเปรียบเทียบ** - เปรียบเทียบข้อมูลก่อนและหลังทำความสะอาด


In [ ]:
# การแสดงผลและ Visualization
print("📊 การแสดงผลและ Visualization:")
print("=" * 50)

# 1. แสดงผลข้อมูลพื้นฐาน
print("1️⃣ ข้อมูลพื้นฐาน:")
print("ข้อมูลต้นฉบับ (5 แถวแรก):")
print(df_dirty.head())
print("\nข้อมูลหลังทำความสะอาด (5 แถวแรก):")
print(df_with_new_cols.head())

# 2. แสดงผลสถิติ
print("\n2️⃣ สถิติพื้นฐาน:")
print("ข้อมูลต้นฉบับ:")
print(df_dirty.describe())
print("\nข้อมูลหลังทำความสะอาด:")
print(df_with_new_cols.describe())

# 3. แสดงผลการกระจายข้อมูล
print("\n3️⃣ การกระจายข้อมูล:")
print("การกระจายของ salary:")
print(f"  ค่าเฉลี่ย: {df_with_new_cols['salary'].mean():.2f}")
print(f"  ค่ากลาง: {df_with_new_cols['salary'].median():.2f}")
print(f"  ค่าเบี่ยงเบนมาตรฐาน: {df_with_new_cols['salary'].std():.2f}")
print(f"  ค่าต่ำสุด: {df_with_new_cols['salary'].min():.2f}")
print(f"  ค่าสูงสุด: {df_with_new_cols['salary'].max():.2f}")

print("\nการกระจายของ age:")
print(f"  ค่าเฉลี่ย: {df_with_new_cols['age'].mean():.2f}")
print(f"  ค่ากลาง: {df_with_new_cols['age'].median():.2f}")
print(f"  ค่าเบี่ยงเบนมาตรฐาน: {df_with_new_cols['age'].std():.2f}")

# 4. แสดงผลการเปรียบเทียบ
print("\n4️⃣ การเปรียบเทียบข้อมูลก่อนและหลังทำความสะอาด:")
print("ขนาดข้อมูล:")
print(f"  ก่อนทำความสะอาด: {df_dirty.shape}")
print(f"  หลังทำความสะอาด: {df_with_new_cols.shape}")

print("\nข้อมูลที่หายไป:")
print("  ก่อนทำความสะอาด:")
for col in df_dirty.columns:
    null_count = df_dirty[col].null_count()
    print(f"    {col}: {null_count}")
print("  หลังทำความสะอาด:")
for col in df_with_new_cols.columns:
    null_count = df_with_new_cols[col].null_count()
    print(f"    {col}: {null_count}")

print("\nข้อมูลซ้ำ:")
print(f"  ก่อนทำความสะอาด: {df_dirty.duplicated().sum()}")
print(f"  หลังทำความสะอาด: {df_with_new_cols.duplicated().sum()}")

# 5. แสดงผลข้อมูลที่จัดกลุ่ม
print("\n5️⃣ ข้อมูลที่จัดกลุ่ม:")
print("ตาม department:")
dept_summary = df_with_new_cols.group_by('department').agg([
    pl.count().alias('count'),
    pl.col('salary').mean().round(2).alias('avg_salary'),
    pl.col('age').mean().round(1).alias('avg_age')
]).sort('department')
print(dept_summary)

print("\nตาม salary_category:")
salary_summary = df_with_new_cols.group_by('salary_category').agg([
    pl.count().alias('count'),
    pl.col('salary').mean().round(2).alias('avg_salary'),
    pl.col('age').mean().round(1).alias('avg_age')
]).sort('salary_category')
print(salary_summary)


## 9️⃣ สรุปและข้อแนะนำ (Summary and Best Practices)

### สรุปขั้นตอนการทำความสะอาดข้อมูล

ในหลักสูตรนี้เราได้เรียนรู้ขั้นตอนการทำความสะอาดข้อมูลอย่างเป็นระบบ:

1. **การเตรียมข้อมูล** - สร้างข้อมูลตัวอย่างที่มีปัญหาต่างๆ
2. **การตรวจสอบข้อมูล** - สำรวจและหาปัญหาในข้อมูล
3. **การจัดการ Missing Values** - แก้ไขข้อมูลที่หายไป
4. **การจัดการข้อมูลซ้ำ** - ลบข้อมูลที่ซ้ำกัน
5. **การจัดการข้อมูลที่ผิดปกติ** - แก้ไขข้อมูลที่ผิดปกติ
6. **การแปลงข้อมูล** - แปลงข้อมูลให้เหมาะสม
7. **การตรวจสอบผลลัพธ์** - ยืนยันความถูกต้องของข้อมูล
8. **การแสดงผล** - แสดงผลข้อมูลเพื่อการตรวจสอบ

### 🎯 ข้อแนะนำที่ดี (Best Practices)

1. **ทำความเข้าใจข้อมูลก่อน** - สำรวจข้อมูลให้เข้าใจก่อนเริ่มทำความสะอาด
2. **บันทึกการเปลี่ยนแปลง** - บันทึกทุกขั้นตอนที่ทำเพื่อการตรวจสอบ
3. **ทดสอบหลายวิธี** - ลองใช้หลายวิธีในการแก้ไขปัญหา
4. **ตรวจสอบผลลัพธ์** - ตรวจสอบผลลัพธ์หลังทำความสะอาดทุกครั้ง
5. **ใช้ข้อมูลตัวอย่าง** - ทดสอบกับข้อมูลตัวอย่างก่อนใช้กับข้อมูลจริง
6. **เก็บข้อมูลต้นฉบับ** - เก็บข้อมูลต้นฉบับไว้เสมอ
7. **ใช้เครื่องมือที่เหมาะสม** - เลือกใช้เครื่องมือที่เหมาะสมกับงาน
8. **ทำงานเป็นทีม** - ทำงานร่วมกับทีมเพื่อตรวจสอบผลลัพธ์

### 🚀 ขั้นตอนต่อไป

หลังจากทำความสะอาดข้อมูลแล้ว สามารถทำขั้นตอนต่อไปได้:

1. **การวิเคราะห์ข้อมูล** - วิเคราะห์ข้อมูลเพื่อหาความรู้
2. **การสร้างโมเดล** - สร้างโมเดลเพื่อทำนายหรือจำแนก
3. **การแสดงผลข้อมูล** - สร้างกราฟและตารางเพื่อแสดงผล
4. **การรายงานผล** - สร้างรายงานผลการวิเคราะห์
5. **การติดตามผล** - ติดตามผลการใช้งานข้อมูล

### 📚 ทรัพยากรเพิ่มเติม

- [Polars Documentation](https://pola.rs/)
- [Data Cleaning Best Practices](https://www.kaggle.com/learn/data-cleaning)
- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)
- [Pandas vs Polars Comparison](https://pola.rs/user-guide/migration/pandas/)

---

**🎉 ขอแสดงความยินดี! คุณได้เรียนรู้การทำความสะอาดข้อมูลด้วย Polars เรียบร้อยแล้ว!**


# 📊 Data Cleaning Tutorial - Hongsa Drilling Database

**ใช้ Polars แทน Pandas**

In [ ]:
import polars as pl
import openpyxl